In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("QUIZ1.ipynb")

ModuleNotFoundError: No module named 'otter'

# Quiz01-Machine Learning

In [30]:
import pandas as pd
import numpy as np

In [31]:
# Load data
data = pd.read_csv('Data1.txt', header=None, names=['Population', 'Profit'])

# Prepare feature matrix and target variable
m = len(data)

# Initialize X, y and theta
x0 = np.ones(m)
population = np.array((data["Population"]))
X = np.array([x0, population]).T
y = np.array(data["Profit"]).reshape(len(data.index), 1)

# Display first few rows
data.head()

,Population,Profit
0,6.1101,17.5920
1,5.5277,9.1302
2,8.5186,13.6620
3,7.0032,11.8540
4,5.8598,6.8233


# Question 1: Implement the Cost Function for Linear Regression
<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />
You are given a dataset (`Data1.txt`) with two columns:
- `Population` (Feature \( X \))
- `Profit` (Target \( y \))

### **Task**
Implement the function `cost_function(X, y, theta)`, which computes the cost function for linear regression.

### **Formula**
$$J(\theta) = \frac{1}{2m} (X\theta - y)^T (X\theta - y)\$$

In [33]:
def cost_function(X, y, theta):
    #m = len(y)
    change = ( X @ theta ) - y
    cost = 1/(2*m) * (change.T) @ ( X @ theta - y)
    return cost
print(cost_function(X, y, np.array([[0], [0]])))

[[32.07273388]]


In [19]:
grader.check("q1")

NameError: name 'grader' is not defined

# Question 2: Implement Gradient Descent
<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

### **Formula for Gradient Update**
$$ \theta := \theta - \frac{\alpha}{m} X^T (X\theta - y)\ $$

Now that we have implemented the cost function in Question 1, we will implement **gradient descent** to minimize $J(\theta)\$.

### **Task**
Implement the function `gradient_descent(X, y, theta, alpha, num_iters)`, which performs **batch gradient descent**.

### **Formula for Gradient Update**
$$ \theta := \theta - \frac{\alpha}{m} X^T (X\theta - y)\ $$

In [27]:
def Batch_gradient_descent(X, y, theta, alpha, num_iters):
    """
    Performs gradient descent to learn theta.

    Parameters:
    X : numpy array of shape (m, n) - Feature matrix
    y : numpy array of shape (m, 1) - Target vector
    theta : numpy array of shape (n, 1) - Parameter vector
    alpha : float - Learning rate
    num_iters : int - Number of iterations

    Returns:
    theta : numpy array of shape (n, 1) - Optimized parameters
    cost_history : list - Cost function values at each iteration
    """
    
    m = len(y)
    cost_history = np.zeros(num_iters)
    for i in range(num_iters):
        gradient = (1 / m) * (X.T) @ ( X @ theta - y)
        theta -= alpha * gradient
        cost = cost_function(X, y, theta)
        cost_history[i] = cost
    
    return theta, cost_history
 
theta, cost_history = Batch_gradient_descent(X, y, np.zeros((2, 1), dtype=np.float64), 0.01, 10)
print(theta)
print(cost_history)

[[0.0086469]
 [0.8007915]]
[6.73719046 5.93159357 5.90115471 5.89522859 5.89009494 5.88500416
 5.87993248 5.87487909 5.86984391 5.86482687]


C:\Users\InaequoSolutions-PC\AppData\Local\Temp\ipykernel_2324\2111401385.py:23: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  cost_history[i] = cost


In [21]:
grader.check("q2")

NameError: name 'grader' is not defined

## Question 3: Features Normalization for Multivariate Linear Regression
<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

When features differ by order of magnitude, performing feature scaling can make **gradient descent** converge much more quickly. Formally:

$$ x := \frac{x - \mu}{\sigma}\ $$

Where:  
- $ \mu $ is the **mean** of the feature.  
- $\sigma$ is the **standard deviation** of the feature.  

🚨 **Important**: Store $ \mu $ and $ \sigma $, as they are needed later for making predictions.

### **Task**
Perform **feature normalization** on the **House Data** features from the dataset `Data2.txt`.

In [28]:
# load data
data = pd.read_csv("Data2.txt", header = None, names=["Size", "Bedrooms","Price"])
m = len(data)

# Initialize X, y and theta
x0 = np.ones(m)
size = np.array((data["Size"]))
bedrooms = np.array((data["Bedrooms"]))
X_mult = np.array([x0, size, bedrooms]).T
y_mult = np.array(data["Price"]).reshape(len(data.index), 1)

data.head()

,Size,Bedrooms,Price
0,2104,3,399900
1,1600,3,329900
2,2400,3,369000
3,1416,2,232000
4,3000,4,539900


In [29]:
def normalize(X):
    """ Normalizes the features in X
    
    returns a normalized version of X where
    the mean value of each feature is 0 and the standard deviation
    is 1. This is often a good preprocessing step to do when
    working with learning algorithms.
    """
    mu = np.zeros(len(X))
    sigma = np.zeros(len(X))
    
    mu = X.mean()
    sigma = X.std()
    X_norm = (X - mu) / sigma
    
    return X_norm, mu, sigma  # Return all values
X_norm, mu, sigma = normalize(X_mult)
print(mu)
print(sigma)

668.2836879432624
1045.7916905133081


In [24]:
grader.check("q3")

NameError: name 'grader' is not defined

## Question 4: Stochastic Gradient Descent
<hr style="border: 5px solid #003262;" />
<hr style="border: 1px solid #fdb515;" />

In this question, you will implement Stochastic Gradient Descent Algorithm with multiple variables to predict the prices of houses. Suppose you are selling your house and you want to know what a good market price would be. One way to do this is to first collect information on recent houses sold and make a model of housing prices.

### **Theory**
Gradient descent is an iterative optimization algorithm used to minimize the cost function:

$$
repeat \ \{ \ \theta_j := \theta_j - \alpha \frac{1}{m}\displaystyle\sum_{i = 1}^{m}(h_\theta(x^{(i)})-y^{(i)})x_j^{(i)}\ \}
$$

For multiple features, this can be vectorized as:

$$
\theta := \theta - \frac{\alpha}{m} X^T (X\theta - \vec{y})
$$

where:  
- $ X $ is the feature matrix,  
- $ y $ is the target vector,  
- $ \theta $ is the parameter vector,  
- $ \alpha $ is the learning rate,  
- $ m $ is the number of training examples.

### **Task**
- Implement the **Stochastic gradient descent** for multiple features.
- The function should support any number of features.
- Return the optimized $ \theta $ values and the cost history.

In [25]:
def Stochastic_gradient_descent(X, y, theta, alpha, num_iters):
    """
    Performs gradient descent to learn theta.

    Parameters:
    X : numpy array of shape (m, n) - Feature matrix
    y : numpy array of shape (m, 1) - Target vector
    theta : numpy array of shape (n, 1) - Parameter vector
    alpha : float - Learning rate
    num_iters : int - Number of iterations

    Returns:
    theta : numpy array of shape (n, 1) - Optimized parameters
    cost_history : list - Cost function values at each iteration
    """
    
    m = len(y)
    cost_history = np.zeros(num_iters * m)
    np.random.seed(1234)  # Fix random seed for reproducibility
    for i in range(num_iters):
        for j in range(m):
            rand_index = np.random.randint(0, m)
            X_j = X[rand_index:rand_index+1]
            y_j = y[rand_index:rand_index+1]
            gradient = X_j.T @ ((X_j @ theta) - y_j)
            theta -= alpha * gradient
            cost_history[i * m + j] = (1/ m) *  np.sum((X @ (theta) - y))
    return theta, cost_history
X_norm, mu, sigma = normalize(X_mult) 
theta, cost_history = Stochastic_gradient_descent(X_norm, y_mult, np.zeros((3, 1), dtype=np.float64), 0.01, 10)
print(theta)
print(cost_history[-1])

[[-102780.69696113]
 [ 159998.84543563]
 [-102487.24970359]]
-5803.440296243021


In [ ]:
grader.check("q4")